<a href="https://colab.research.google.com/github/aliiamrr/EGX_Kaggle_Processing/blob/main/Stock_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/ABUK.csv')

In [ ]:
print(df.head())
print(df.info())

         Date  Price   Open   High    Low     Vol. Change %
0  08/09/2023  48.60  47.80  48.89  47.50    1.02M    1.59%
1  08/08/2023  47.84  47.93  48.12  47.30  606.90K   -0.15%
2  08/07/2023  47.91  47.61  48.19  47.47  653.90K    0.97%
3  08/06/2023  47.45  46.99  47.49  46.60  453.75K    1.09%
4  08/03/2023  46.94  46.75  46.94  46.15  368.86K    0.58%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1116 entries, 0 to 1115
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      1116 non-null   object 
 1   Price     1116 non-null   float64
 2   Open      1116 non-null   float64
 3   High      1116 non-null   float64
 4   Low       1116 non-null   float64
 5   Vol.      1116 non-null   object 
 6   Change %  1116 non-null   object 
dtypes: float64(4), object(3)
memory usage: 61.2+ KB
None


In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format = '%m/%d/%Y')

In [ ]:
def convert_volume_to_number(volume_str):
    """
    Convert volume string with K, M, B, T suffixes to integers.
    Examples:
    '1.02M' -> 1020000
    '606.90K' -> 606900
    '1.5B' -> 1500000000
    """
    if pd.isna(volume_str) or volume_str == '' or volume_str == '-':
        return 0

    # Remove commas and strip whitespace
    volume_str = str(volume_str).replace(',', '').strip()

    # Multipliers dictionary
    multipliers = {
        'K': 1_000,
        'M': 1_000_000,
        'B': 1_000_000_000,
        'T': 1_000_000_000_000
    }

    # Check if last character is a known multiplier
    if volume_str[-1].upper() in multipliers:
        multiplier = multipliers[volume_str[-1].upper()]
        number_part = float(volume_str[:-1])
        return int(number_part * multiplier)
    else:
        # If there is no suffix, we will assume its a raw number
        try:
            return int(float(volume_str))
        except ValueError:
            return 0

In [ ]:
# Apply the function/transformation to the volume column
df['Vol.'] = df['Vol.'].apply(convert_volume_to_number)

In [ ]:
print(df.head())

        Date  Price   Open   High    Low     Vol. Change %
0 2023-08-09  48.60  47.80  48.89  47.50  1020000    1.59%
1 2023-08-08  47.84  47.93  48.12  47.30   606900   -0.15%
2 2023-08-07  47.91  47.61  48.19  47.47   653900    0.97%
3 2023-08-06  47.45  46.99  47.49  46.60   453750    1.09%
4 2023-08-03  46.94  46.75  46.94  46.15   368860    0.58%


In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1116 entries, 0 to 1115
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      1116 non-null   datetime64[ns]
 1   Price     1116 non-null   float64       
 2   Open      1116 non-null   float64       
 3   High      1116 non-null   float64       
 4   Low       1116 non-null   float64       
 5   Vol.      1116 non-null   int64         
 6   Change %  1116 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 61.2+ KB
None


In [ ]:
df = df.rename(columns={'Vol.': 'Volume', 'Price': 'Close'})

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1116 entries, 0 to 1115
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      1116 non-null   datetime64[ns]
 1   Close     1116 non-null   float64       
 2   Open      1116 non-null   float64       
 3   High      1116 non-null   float64       
 4   Low       1116 non-null   float64       
 5   Volume    1116 non-null   int64         
 6   Change %  1116 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 61.2+ KB
None


In [ ]:
df = df.drop('Change %', axis = 1)

In [ ]:
monthly_df = (
    df.resample('ME', on='Date')
      .agg({
          'Open': 'first',
          'High': 'max',
          'Low': 'min',
          'Close': 'last',
          'Volume': 'sum'
      })
      .reset_index()
)

In [ ]:
print(monthly_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    56 non-null     datetime64[ns]
 1   Open    56 non-null     float64       
 2   High    56 non-null     float64       
 3   Low     56 non-null     float64       
 4   Close   56 non-null     float64       
 5   Volume  56 non-null     int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 2.8 KB
None


In [ ]:
print(monthly_df.head())

        Date   Open   High    Low  Close   Volume
0 2019-01-31  22.95  24.50  22.40  23.30  2518019
1 2019-02-28  23.55  27.02  22.80  25.03  3882270
2 2019-03-31  24.70  29.25  24.57  26.83  3660049
3 2019-04-30  26.30  27.90  26.00  26.72   910690
4 2019-05-31  26.54  26.54  23.50  23.81   677830


In [ ]:
# Downloading the df as a csv
monthly_df.to_csv('ABUK_Kaggle_Cleaned', index=False)